<a href="https://colab.research.google.com/github/vitroid/GenIce-core/blob/main/example-doping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## for Google Colaboratory


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/genice-dev/GenIce-core.git pairlist
except:
    pass

## 1. Prepare an undirected graph.


In [ ]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go


# a useful function to draw a graph with plotly.
def draw_graph(g: nx.Graph, pos: dict, fixed: list = None, dopant: list = []):
    # draw the graph with edges and labels in 3D
    pos = np.array([pos[i] for i in sorted(g.nodes())])

    # 通常の辺のリスト
    normal_edges = [
        (i, j)
        for i, j in g.edges()
        if fixed is None or [i, j] not in fixed and [j, i] not in fixed
    ]
    # 固定された辺のリスト
    fixed_edges = fixed if fixed is not None else []

    normal_nodes = [i for i in g.nodes() if i not in dopant]
    dopant_nodes = [i for i in dopant]

    normal_pos = pos[normal_nodes]
    dopant_pos = pos[dopant_nodes]

    fig = go.Figure(
        data=[
            # ノードの表示
            go.Scatter3d(
                x=normal_pos[:, 0],
                y=normal_pos[:, 1],
                z=normal_pos[:, 2],
                mode="markers+text",
                marker=dict(size=2, color="blue"),
                text=[str(i) for i in normal_nodes],
                textposition="top center",
            ),
            go.Scatter3d(
                x=dopant_pos[:, 0],
                y=dopant_pos[:, 1],
                z=dopant_pos[:, 2],
                mode="markers+text",
                marker=dict(size=4, color="red"),
                text=[str(i) for i in dopant_nodes],
                textposition="top center",
            ),
            # 通常の辺の表示
            *[
                go.Scatter3d(
                    x=[pos[i, 0], pos[j, 0]],
                    y=[pos[i, 1], pos[j, 1]],
                    z=[pos[i, 2], pos[j, 2]],
                    mode="lines",
                    line=dict(color="gray", width=2),
                    hoverinfo="none",
                )
                for i, j in normal_edges
            ],
            # 固定された辺（矢印）の表示
            *[
                go.Scatter3d(
                    x=[pos[i, 0], pos[j, 0]],
                    y=[pos[i, 1], pos[j, 1]],
                    z=[pos[i, 2], pos[j, 2]],
                    mode="lines",
                    line=dict(color="green", width=3),
                    hoverinfo="none",
                )
                for i, j in fixed_edges
            ],
            # 矢印の先端（コーン）の表示
            *[
                go.Cone(
                    x=[(pos[j, 0] - pos[i, 0]) * 0.8 + pos[i, 0]],
                    y=[(pos[j, 1] - pos[i, 1]) * 0.8 + pos[i, 1]],
                    z=[(pos[j, 2] - pos[i, 2]) * 0.8 + pos[i, 2]],
                    u=[(pos[j, 0] - pos[i, 0]) * 0.4],
                    v=[(pos[j, 1] - pos[i, 1]) * 0.4],
                    w=[(pos[j, 2] - pos[i, 2]) * 0.4],
                    colorscale=[[0, "green"], [1, "green"]],
                    showscale=False,
                )
                for i, j in fixed_edges
            ],
        ]
    )
    fig.show()

Here we prepare a dodecahedral graph plus one extra node at the center. The nodes 0, 6, 13, and 17 are four-connected and others are 3-connected (defect or surface nodes).


In [ ]:
from logging import getLogger, DEBUG

logger = getLogger()
logger.setLevel(DEBUG)

g = nx.dodecahedral_graph()  # dodecahedral 20mer and an additional node at the center
pos = nx.spring_layout(g, dim=3)
print(pos)

# add a new node at the center
g.add_node(20)
pos[20] = (0, 0, 0)
g.add_edge(0, 20)
g.add_edge(6, 20)
g.add_edge(13, 20)
g.add_edge(17, 20)

# pos = nx.spring_layout(g, pos=pos)

draw_graph(g, pos)

Now let's assume that the node 0 is a cation and donating hydrogen bonds to its neighbors.


In [ ]:
cation = 0

# prefixed directed edges.
fixed = []
for node in g.neighbors(cation):
    fixed.append([cation, node])

draw_graph(g, pos, dopant=[cation], fixed=fixed)

Other edges are arranged appropriately by the GenIce-core.


In [ ]:
import genice_core

# set orientations of the hydrogen bonds.
dg = genice_core.ice_graph(g, vertexPositions=pos, fixedEdges=nx.DiGraph(fixed))

draw_graph(g, pos, fixed=dg.edges(), dopant=[cation])